In [3]:
############ IMPORTS ####################
import sys
sys.path.append("../")
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Flatten, SimpleRNN
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical as keras_to_categorical

############# CONFIGS ##########################

os.environ["CUDA_VISIBLE_DEVICES"]="3"

datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 128
EPOCH = 4
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(feat_size,seq, Classes):
    func = SimpleRNN
    clf = Sequential()
    clf.add(func(64, return_sequences=True, input_shape=(feat_size, seq)))
    clf.add(func(64, return_sequences=True))
    clf.add(Flatten())
    clf.add(Dense(Classes, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return clf
        
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#     TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#     TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    
    TrainPatch1 = TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1]*TrainPatch1.shape[2],TrainPatch1.shape[3])
    TestPatch1 = TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3])
    
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
        
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))

    for iter in range(3):
        n_bands, sequences = TrainPatch1.shape[1:]
        clf = get_model_compiled(n_bands, sequences,Classes)
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(), Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(), Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])
        
        
        clf = load_model(datasetName+"/best_model_HSIOnly.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)
        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc
    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/RNN_Report_' + datasetName +'.txt')


----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 121, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 121, 63])
Test label shape =  torch.Size([1, 29395])
Epoch 1/4
7/7 [==============================] - 9s 1s/step - loss: 1.9844 - accuracy: 0.3346 - val_loss: 0.9623 - val_accuracy: 0.5573
Epoch 2/4
7/7 [==============================] - 8s 1s/step - loss: 0.8497 - accuracy: 0.6838 - val_loss: 0.6518 - val_accuracy: 0.7959
Epoch 3/4
7/7 [==============================] - 8s 1s/step - loss: 0.4151 - accuracy: 0.8486 - val_loss: 0.4067 - val_accuracy: 0.8302
Epoch 4/4
7/7 [==============================] - 8s 1s/step - loss: 0.2636 - accuracy: 0.8816 - val_loss: 0.4438 - val_accuracy: 0.8230
Epoch 1/4
7/7 [==============================] - 9s 1s/step - loss: 1.3516 - accuracy: 0.5177 - val_loss: 0.8289 - val_accuracy: 0.6975
Epoch 2/4
7/7 [========================